# Clovek ne jezi se (aka Mensch aergere dich nicht)



## Agents

Agents are a container for different game policies.


In [1]:
import numpy as np
from random import randint

from clovek_ne_jezi_se.game import Board, Game, Move
from clovek_ne_jezi_se.agent import Player, FurthestAlongAgent

## Game

### Game state

The game state is represented by a triple of arrays: $(waiting, spaces, homes)$.

In [2]:
symbols = ['red', 'yellow', 'green', 'blue']
players = []
for symbol in symbols:
    player = Player(symbol=symbol, number_of_players=4)
    player.initialize_home()
    players.append(player)
game = Game(players)
game.initialize()

# Empty waiting area of one symbol
# TODO: prohibit (or make private) such non-conserving moves, see issue #6
game.set_waiting_count_array('yellow', count=0)





The waiting area is represented by the counts per player.

In [3]:
game.board.waiting_count

{'red': 4, 'yellow': 0, 'green': 4, 'blue': 4}

The spaces (i.e. main board) are represented by a 1-d array.

In [4]:
# Occupy one start position
occupied_position = game.get_player('green').get_start()
game.set_space_array('blue', occupied_position)
game.get_spaces_array()

array([-1, -1, -1, -1, -1, -1, -1, -1,  3, -1, -1, -1, -1, -1, -1, -1])

And the home area is represented by a (number of players) x (number of tokens / player) array.

In [5]:
# Set some home spaces
game.set_homes_array('green', 0)
game.set_homes_array('green', 1)
game.get_homes_array()

array([[-1, -1, -1, -1],
       [-1, -1, -1, -1],
       [ 2,  2, -1, -1],
       [-1, -1, -1, -1]])

The `Board` class has more human-readable representation:

In [6]:
game.board


    -------------
    | - | - | blue |
----------------------
| - | - |    | - | - |
--------      -------|
| - |            | - |
--------      -------|
| - | - |    | - | - |
----------------------
    | - | - | - |
    -------------
player red pieces in waiting area: 4 
player red home: - | - | - | -

player yellow pieces in waiting area: 0 
player yellow home: - | - | - | -

player green pieces in waiting area: 4 
player green home: green | green | - | -

player blue pieces in waiting area: 4 
player blue home: - | - | - | -

### Game moves and actions

Given a dice roll, an action is a choice among valid moves of

* *leave waiting*: From waiting area to player start space upon rolling a 6
* *space advance*: Move within spaces (main playing board)
* *space to home*: Move from spaces to player's home area
* *home advance*: Move within player's home area

Validity checks are implemented for each of these (e.g. the end position must be unoccupied).

Moves are created in `Game.move_factory`, which includes validation checks.

In [7]:
# Valid move
game.move_factory(symbol='red', kind='leave_waiting', roll=6)

Move(symbol='red', kind='leave_waiting', roll=6, start=None)

In [13]:
# Invalid move: May only leave waiting area if a 6 is rolled
try: 
    game.move_factory(symbol='red', kind='leave_waiting', roll=5)
except ValueError as err:
    print(err)

# Invalid move: Advancing beyond "pre-home" position is space_to_home, not space_advance
try:
    game.move_factory(
        symbol='red', kind='space_advance', roll=1, start=game.get_player('red').get_prehome_position()
    )
except ValueError as err:
    print(err)

Invalid move entered
Invalid move entered
